In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# First Step: Loading Images

## 1- images path:

In [2]:
data_path = '/kaggle/input/drone-detection/tensorflow obj detection drone'

In [3]:
import os
import cv2

## 2- loading train Data

In [4]:
def load_images_and_labels(folder_path, target_size=(256, 256)):
    
    images = []
    labels = []

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            label = filename.split('_')[1] 
            labels.append(label)
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img_resized = cv2.resize(img, target_size)
                images.append(img_resized)
            else:
                print(f"Warning: Could not load image {filename}")

    return images, labels

In [5]:
train_folder =data_path + "/train"
images, labels = load_images_and_labels(train_folder)

In [6]:
len(images) , len(labels)

(10799, 10799)

In [7]:
valid_folder = data_path + '/valid'
valid_images , valid_labels = load_images_and_labels(valid_folder)

In [8]:
len(valid_images) , len(valid_labels)

(603, 603)

# Data Normalization

In [9]:
def normalize_images(images):
    normalized_images = [img / 255.0 for img in images]
    return normalized_images

In [10]:
normalized_images = normalize_images(images)

In [11]:
normalized_images_valid = normalize_images(valid_images)

# Label Encoding

In [12]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [13]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(labels)
y_val_encoded = label_encoder.transform(valid_labels)

y_train_one_hot = to_categorical(y_train_encoded)
y_val_one_hot = to_categorical(y_val_encoded)

# Building a model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adamax

In [15]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(optimizer=Adamax(learning_rate = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
from tensorflow.keras.utils import Sequence

In [18]:
class DataGenerator(Sequence):
    def __init__(self, images, labels, batch_size):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_images = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = np.array(batch_images)
        batch_labels = np.array(batch_labels)

        return batch_images, batch_labels

In [19]:
train_generator = DataGenerator(normalized_images, y_train_one_hot, batch_size=32)

In [20]:
val_generator = DataGenerator(normalized_images_valid, y_val_one_hot, batch_size=32)

In [21]:
model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


338/338 ━━━━━━━━━━━━━━━━━━━━ 667s 2s/step - accuracy: 0.5597 - loss: 1.5026 - val_accuracy: 0.8474 - val_loss: 0.4058
Epoch 2/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 698s 2s/step - accuracy: 0.8145 - loss: 0.4421 - val_accuracy: 0.8789 - val_loss: 0.2997
Epoch 3/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 684s 2s/step - accuracy: 0.8498 - loss: 0.3601 - val_accuracy: 0.8939 - val_loss: 0.2410
Epoch 4/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 738s 2s/step - accuracy: 0.8872 - loss: 0.2773 - val_accuracy: 0.9071 - val_loss: 0.2091
Epoch 5/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 678s 2s/step - accuracy: 0.8994 - loss: 0.2432 - val_accuracy: 0.9254 - val_loss: 0.1954
Epoch 6/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.9115 - loss: 0.2115 - val_accuracy: 0.9370 - val_loss: 0.1887
Epoch 7/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 687s 2s/step - accuracy: 0.9241 - loss: 0.1948 - val_accuracy: 0.9353 - val_loss: 0.1656
Epoch 8/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 676s 2s/step - accuracy: 0.9249 - loss: 0.1725 - val_accuracy: 0.950

# Test Data

In [22]:
test_folder = data_path + '/test'
test_images , test_labels = load_images_and_labels(test_folder)

In [23]:
normalized_images_test = normalize_images(test_images)

In [24]:
y_test_encoded = label_encoder.transform(test_labels)
y_test_one_hot = to_categorical(y_test_encoded)

In [25]:
test_generator = DataGenerator(normalized_images_test, y_test_one_hot, batch_size=32)

In [26]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 505ms/step - accuracy: 0.9730 - loss: 0.0833
Test Accuracy: 0.95


## this model is for basic classification